# PARCIAL 1 TAM (Teoría de Aprendizaje de Máquina)




## Ej 1 


Dado el modelo de regresion : 

tn = 
𝜙
(
𝑥
𝑛
)
⊤
𝑤
+
𝜂
𝑛
t 
n
​
 =ϕ(x 
n
​
 ) 
⊤
 w+η 
n
​
 , con 
𝜂
𝑛
∼
𝑁
(
0
,
𝜎
2
)
η 
n
​
 ∼N(0,σ 
2
 )


Presentamos la inferencia o problema de optimización para los siguientes casos asumiendo datos i.i.d : 


### MÍNIMOS CUADRADOS (OLS)

### MÍNIMOS CUADRADOS REGULARIZADOS 

### MÁXIMA VEROSIMILITUD 

### MÁXIMO A POSTERIORI (MAP)

### MODELO BAYESIANO LINEAL GAUSSIANO 

### REGRESIÓN RÍGIDA KERNEL

### PROCESOS GAUSIANOS (GP)

### Finalmente discutimos similitudes y diferencias:  

## EJ 2

Presentamos un cuadro comparativo de los siguientes regresores, en donde se discute el modelo matem atico, funcion de costo, estrategia
de optimizacion, relacion con los esquemas b́asicos de regresion discutidos en el anterior ejercicio y tambien su escalabilidad en caso de ser util:


• LinearRegresor
• Lasso
• ElasticNet
• KernelRidge
• SGDRegressor
• BayesianRidge
• Gaussian Process Regressor
• Support Vector Machines Regressor
• RandomForestRegressor

• GradientBoostingRegressor
• XGBoost


## EJ 3

Verificamos en que consiste y el funcionamiento principal de la librer ́ıa RAPIDS. Posteriormente, se elabora una tabla que compara
los metodos e hiperparametros mas relevantes de los regresores mencionados en el anterior ejercicio, indicando su implementacion o el
algoritmo equivalente disponible en RAPIDS.


## EJ 4

Finalmente, se presenta un analisis detallado del conjunto de datos disponible en https://www.kaggle.com/competitions/nfl-big-data-bowl-2026-
prediction/overview. Incluya la descripci on de las variables de entrada y salida, el objetivo principal del concurso, un analisis
exploratorio basico, los procedimientos de codificaci on de valores faltantes y variables categoricas, ası como las estrategias de
ingenierıa de caracterısticas —si aplican—. Finalmente, proporcione todos los detalles necesarios para comprender, interpretar y
analizar adecuadamente el conjunto de datos.
Luego, utilizando un esquema de validacion hold-out con una partici  ́on del 60 % para entrenamiento, 20 % para validacion y 20 %
para evaluacion del rendimiento, compare los regresores mencionados en el punto 2 —considerando sus posibles implementacio-
nes en RAPIDS, segun lo indicado en el punto 3—, empleando el conjunto de datos de entrenamiento disponible en el dataset.
Realice las consideraciones necesarias para la implementacion y sintonizacion de cada modelo mediante optimizacion bayesiana,
con el fin de determinar los hiperparametros mas relevantes de cada algoritmo. Justifique la seleccion de los hiperparametros a
optimizar, ası como la rejilla o los rangos de valores definidos para cada modelo, de acuerdo con las caracterısticas estudiadas
y el score a minimizar. Finalmente, presente los rendimientos promedio obtenidos sobre el conjunto de evaluacion, junto con su
respectiva desviacion est ́andar, empleando las siguientes metricas de desempeno: MAE, MSE, R2 y MAPE

# END NB
Cristhian Mateo Almeida Gómez



presentado al profesor Andrés Marino Alvarez Meza. 